In [ ]:
pip install langchain openai faiss-cpu gradio tiktoken sentence-transformers


In [ ]:
pip install -U langchain-community


In [ ]:
import zipfile
import os
from langchain.document_loaders import TextLoader, DirectoryLoader

# Path to your zip file
zip_path = "/content/archive (16).zip"
extract_path = "unzipped_data"

# Extract zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Load all .txt files from unzipped folder
loader = DirectoryLoader(extract_path, glob="**/*.txt", loader_cls=TextLoader)
documents = loader.load()


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(documents, embedding)


In [ ]:
pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI  # updated import
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import os

# Set API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Initialize LLM & memory
llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Build QA Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
    memory=memory,
    return_source_documents=True
)


In [ ]:
pip install streamlit langchain langchain-community langchain-openai faiss-cpu


In [ ]:
OPENAI_API_KEY = "your-api-key-here"


In [ ]:
# sotu_chatbot.py
import streamlit as st
import zipfile
import os
import tempfile

from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

# 🔑 Set your API key here
OPENAI_API_KEY = "your-api-key-here"

st.set_page_config(page_title="State of the Union Chatbot", layout="centered")
st.title("📜 State of the Union RAG Chatbot (1790–2018)")

uploaded_zip = st.file_uploader("📂 Upload ZIP file of .txt documents", type="zip")

if "qa_chain" not in st.session_state:
    st.session_state.qa_chain = None
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

if uploaded_zip and st.session_state.qa_chain is None:
    with tempfile.TemporaryDirectory() as tmpdir:
        zip_path = os.path.join(tmpdir, "data.zip")
        with open(zip_path, "wb") as f:
            f.write(uploaded_zip.getvalue())

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(tmpdir)

        st.info("🔍 Loading and indexing documents...")

        loader = DirectoryLoader(tmpdir, glob="**/*.txt", loader_cls=TextLoader)
        documents = loader.load()

        embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        db = FAISS.from_documents(documents, embedding)

        llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=db.as_retriever(),
            memory=memory,
            return_source_documents=True
        )

        st.session_state.qa_chain = qa_chain
        st.success("✅ Chatbot is ready!")

# Chat interface
if st.session_state.qa_chain:
    with st.chat_message("assistant"):
        st.markdown("Hello! Ask about any U.S. presidential speech 🇺🇸")

    user_input = st.chat_input("Ask a question...")
    if user_input:
        with st.chat_message("user"):
            st.markdown(user_input)

        result = st.session_state.qa_chain({
            "question": user_input,
            "chat_history": st.session_state.chat_history
        })

        st.session_state.chat_history.append((user_input, result["answer"]))
        with st.chat_message("assistant"):
            st.markdown(result["answer"])


2025-07-21 18:02:10.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 18:02:10.373 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 18:02:10.378 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 18:02:10.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 18:02:10.384 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 18:02:10.384 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 18:02:10.394 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-21 18:02:10.394 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
pip install streamlit langchain langchain-community langchain-openai faiss-cpu sentence-transformers


In [ ]:
!pip install streamlit langchain langchain-community langchain-openai faiss-cpu sentence-transformers
!streamlit run sotu_chatbot.py & npx localtunnel --port 8501


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: sotu_chatbot.py
⠙⠹⠸⠼⠴⠦⠧Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) ok

⠙npm error canceled
⠙npm error A complete log of this run can be found in: /root/.npm/_logs/2025-07-21T18_02_38_906Z-debug-0.log
⠙